# Обработка признаков

В этом домашнем задании вы будете решать задачу предсказания стоимости автомобилей по их различным характеристикам.

In [557]:
import pandas as pd

RANDOM_STATE = 42

In [558]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/cars_prices.csv", decimal='.')

In [559]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


### Описание некоторых признаков

`symboling` - rating corresponds to the degree to which the auto is more risky than its price indicates (+3 more risk and -3 is pretty safe)  
`make` - car types (i.e. car brand)  
`fuel-type` - types of fuel (gas or diesel)  
`aspiration` - engine aspiration (standard or turbo)  
`num-of-doors` - numbers of doors (two or four)  
`body-style` - car body style (sedan or hachback)  
`drive-wheels` - which types of drive wheel (forward-fwd, reversed-rwd)  
`engine-location` - engine mounted location (front or back)  
`wheel-base` - расстояние между осями передних и задних колес  
`length` - car lenght  
`weight` - car weight  
`width` - car width  
`height` - car height  

In [560]:
df.shape

(205, 26)

In [561]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

## Заполнение пропусков

Пропуски в этом датасете обозначены как `?`

In [562]:
for c in df.columns:
    print(c, len(df[df[c] == '?']))

symboling 0
normalized-losses 41
make 0
fuel-type 0
aspiration 0
num-of-doors 2
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 4
stroke 4
compression-ratio 0
horsepower 2
peak-rpm 2
city-mpg 0
highway-mpg 0
price 4


Удалите строки, для которых неизвестно значение price, так как это целевая переменная.

In [563]:
df=df[df['price']!='?']

## Вопрос для Quiz

Сколько строк осталось в данных?

In [564]:
df.shape

(201, 26)

Заполните средним значением пропуски в столбцах для числовых признаков и самым популярным значением для категориальных признаков
* `num-of-doors`
* `bore`
* `stroke`
* `horsepower`
* `peak-rpm`

In [565]:
peak=df['peak-rpm'][df['peak-rpm']!='?']
peak.astype(int).mean()

5117.587939698493

In [567]:
numeric = df.columns[(df.dtypes == 'int64') | (df.dtypes == 'float64')].tolist()
categorical=['num-of-doors','bore','stroke','horsepower','peak-rpm']
features=numeric+categorical
for i in features:
    if i in numeric:
        df.loc[df[i]=='?',i]=df[i].mean()
    else:
        top=df[i].value_counts().idxmax()
        df.loc[df[i]=='?',i]=top


## Вопрос для Quiz

Чему равно среднее значение `peak-rpm` до заполнения пропусков? Ответ округлите до целого числа.

5118

Пропуски в столбце `normalized-losses` предскажите при помощи линейной регрессии по признакам
`symboling`, `wheel-base`, `length`, `width`, `height`, `curb-weight`, `engine-size`, `compression-ratio`, `city-mpg`, `highway-mpg` и заполните их предсказаниями

In [569]:
from sklearn.linear_model import LinearRegression

features=['symboling','wheel-base','length', 'width', 'height', 'curb-weight', 
          'engine-size','compression-ratio','city-mpg','highway-mpg']
y=df['normalized-losses'][df['normalized-losses']!='?']

X_train=df[features][df['normalized-losses']!='?']
X_test=df[features][df['normalized-losses']=='?']

model= LinearRegression()
model.fit(X_train,y)
y_pred=model.predict(X_test)

In [570]:
df.loc[df['normalized-losses']=='?','normalized-losses']=y_pred

## Вопрос для Quiz

Чему равно предсказание линейной регрессии на первом пропущенном значении? Ответ округлите до целого числа.

168

## 2. Кодирование категориальных признаков

1. Закодируйте бинарные признаки `fuel-type`, `aspiration`, `num-of-doors`, `engine-location` каждый отдельной колонкой, состоящей из 0 и 1.
Единицей кодируйте самую частую категорию.

In [571]:
categor=['fuel-type','aspiration','num-of-doors','engine-location']

for i in categor:
    top=df[i].value_counts().idxmax()
    df[i]=(df[i] == top).astype(int)

2. Вынесите в переменную `y` целевую переменную `price`, а все остальные колонки - в матрицу `X`.

Закодируйте признаки `make`, `body-style`, `engine-type`, `fuel-system` при помощи LeaveOneOutEncoder.

**Дальше все время работайте с объектами `X`, `y`.**

In [572]:
!pip install category_encoders -q

In [573]:
df['price']=df['price'].astype(int)

In [574]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

feat=['make', 'body-style', 'engine-type', 'fuel-system']

X = df.drop(columns='price')
y = df['price']

# X= pd.DataFrame(X)
# y= pd.Series(y)


encoder = LeaveOneOutEncoder()
X_encoded = encoder.fit_transform(X[feat],y)

X=pd.concat([X.drop(columns=feat),X_encoded],axis=1)

## Вопрос для Quiz

Чему равно среднее значение в столбце `body-style` после кодирования? Ответ округлите до целого числа.

3. Закодируйте признак `drive-wheels` при помощи OHE из библиотеки category_encoders.

In [576]:
from category_encoders.one_hot import OneHotEncoder

model=OneHotEncoder()
X_encoded=model.fit_transform(df['drive-wheels'])

X=pd.concat([X.drop(columns='drive-wheels'),X_encoded],axis=1)

4. В столбце `num-of-cylinders` категории упорядочены по смыслу. Закодируйте их подряд идущими числами, начиная с 1, согласно смыслу.

Подряд идущими числами означает - 1, 2, 3 и так далее без пропусков.

In [577]:
X.columns

Index(['symboling', 'normalized-losses', 'fuel-type', 'aspiration',
       'num-of-doors', 'engine-location', 'wheel-base', 'length', 'width',
       'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'bore',
       'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'make', 'body-style', 'engine-type', 'fuel-system',
       'drive-wheels_1', 'drive-wheels_2', 'drive-wheels_3'],
      dtype='object')

In [578]:
cylinder_mapping = {
    'two': 1,
    'three': 2,
    'four': 3,
    'five': 4,
    'six': 5,
    'eight': 6,
    'twelve': 7
}

X['num-of-cylinders'] = X['num-of-cylinders'].map(cylinder_mapping)

## Вопрос для Quiz

Сколько столбцов получилось в матрице `X`?

In [579]:
X.shape

(201, 27)

In [580]:
X['normalized-losses'].head()

0    168.072493
1    168.072493
2    134.001799
3           164
4           164
Name: normalized-losses, dtype: object

In [585]:
X['normalized-losses'] = X['normalized-losses'].astype(float)
X['bore'] = X['bore'].astype(float)
X['stroke'] = X['stroke'].astype(float)
X['horsepower'] = X['horsepower'].astype(float)
X['peak-rpm'] = X['peak-rpm'].astype(float)

y = y.astype(float)

Разбейте данные на тренировочную и тестовую часть в пропорции 3 к 1, зафиксируйте random_state = 42.

In [586]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state = 42)

Масштабируйте данные при помощи MinMaxScaler.

Обучайте масштабирование на тренировочных данных, а потом примените и к трейну, и к тесту.

In [587]:
from sklearn.preprocessing import MinMaxScaler

min_max=MinMaxScaler()
min_max.fit(X_train)
X_train_prep=min_max.transform(X_train)
X_test_prep=min_max.transform(X_test)


Обучите на тренировочных данных линейную регрессию, сделайте предсказание на тесте и вычислите значение $R^2$ на тестовых данных.

In [588]:
from sklearn.metrics import r2_score

model=LinearRegression()
model.fit(X_train_prep,y_train)
y_pred=model.predict(X_test_prep)

r2 = r2_score(y_test, y_pred)
r2

0.9068485682379008

## Вопрос для Quiz

Чему равно значение $R^2$ на тестовых данных? Ответ округлите до сотых.